In [154]:
#Dependencies
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import balanced_accuracy_score, r2_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import tensorflow as tf

In [155]:
#read csv into df
df = pd.read_csv("../Levels_Fyi_Salary_Data 2.csv")
# columns to drop... salary_currency, salary, work_year
df.sample(25)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
39436,1/14/2021 22:26:46,Amazon,L6,Technical Program Manager,215000,"Seattle, WA",11.0,2.0,DevOps,133000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree
4102,2/4/2019 22:42:07,Microsoft,60,Software Engineer,128000,"Redmond, WA",3.0,3.0,Web Development (Front-End),0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
61876,9/16/2018 9:56:05,Amazon,L7,Software Engineering Manager,402000,"Seattle, WA",19.0,7.0,Full Stack,160000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
26666,8/15/2020 21:14:04,Intel Corporation,Grade 9,Software Engineer,226000,"Chandler, AZ",15.0,15.0,Distributed Systems (Back-End),165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
17948,3/6/2020 10:37:42,Google,L3,Technical Program Manager,177000,"Sunnyvale, CA",5.0,2.0,UX,116000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
49867,4/28/2021 10:56:05,Teradata,L2,Software Engineer,105000,"San Diego, CA",4.0,4.0,Cloud,98000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2587,11/5/2018 21:25:48,Splunk,P3,Software Engineer,157000,"Boulder, CO",3.0,0.0,Distributed Systems (Back-End),120000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
48757,4/18/2021 18:52:36,Uber,L5B,Software Engineer,485000,"New York, NY",6.0,0.0,Mobile (iOS + Android),222000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
51450,5/14/2021 10:38:58,Facebook,E5,Software Engineer,424000,"Seattle, WA",11.0,2.0,ML / AI,210000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
41243,2/4/2021 19:15:54,Amazon,L6,Recruiter,245000,"New York, NY",10.0,1.0,Operations,145000.0,...,0,0,0,0,1,0,0,0,White,Bachelor's Degree


In [156]:
df.nunique()


timestamp                  62561
company                     1631
level                       2923
title                         15
totalyearlycompensation      893
location                    1050
yearsofexperience             65
yearsatcompany                81
tag                         3058
basesalary                   482
stockgrantvalue              612
bonus                        335
gender                         4
otherdetails               12841
cityid                      1045
dmaid                        149
rowNumber                  62642
Masters_Degree                 2
Bachelors_Degree               2
Doctorate_Degree               2
Highschool                     2
Some_College                   2
Race_Asian                     2
Race_White                     2
Race_Two_Or_More               2
Race_Black                     2
Race_Hispanic                  2
Race                           5
Education                      5
dtype: int64

In [157]:
titles_df = df[df["title"]=="Data Scientist"]
titles_df


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",4.0,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",11.0,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",10.0,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",4.0,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",8.0,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,3/26/2019 17:25:02,IBM,8,Data Scientist,155000,"Washington, DC",5.0,2.0,ML / AI,141000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62283,6/17/2018 21:07:18,Facebook,E3,Data Scientist,150000,"Singapore, SG, Singapore",0.0,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62285,7/16/2018 20:11:40,ADP,Sr UX Researcher 4,Data Scientist,185000,"Roseland, NJ",8.0,2.0,UX Research,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62529,6/12/2018 20:54:06,Google,T6,Data Scientist,685000,"Kirkland, WA",22.0,2.0,ML / AI,221000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [196]:
#Binning years of experience for better encoding
titles_df['yearsofexperience'].value_counts()



2.0     351
5.0     343
3.0     314
4.0     301
1.0     214
0.0     190
6.0     178
7.0     141
10.0    132
8.0     124
9.0      64
15.0     45
12.0     42
11.0     37
13.0     25
20.0     21
16.0     18
14.0     15
17.0      6
18.0      4
25.0      4
23.0      2
22.0      2
19.0      1
26.0      1
30.0      1
21.0      1
34.0      1
Name: yearsofexperience, dtype: int64

In [197]:
# # Replace in dataframe
# years_labels = []
# for years in titles_df['yearsofexperience']:
#     if years == 0.0:
#         years = "Entry Level"
#     elif years < 5.0:
#         years = "Early Career"
#     elif (years>=5.0 and years <10.0):
#         years = "Mid level"
#     else:
#         years = "Senior"
#     years_labels.append(years)
# years_labels

In [198]:
#reassign company column to bin name
year_level_df = titles_df.copy()
# year_level_df['yearsofexperience'] = years_labels
year_level_df.sample(20)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
5197,3/24/2019 15:03:17,Google,L4,Data Scientist,200000,"Mountain View, CA",0.0,0.0,ML / AI,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
23053,6/25/2020 2:04:43,TD Bank,L10,Data Scientist,128000,"Toronto, ON, Canada",3.0,3.0,ML / AI,110000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
12751,11/5/2019 16:13:47,LinkedIn,Senior,Data Scientist,256000,"San Francisco, CA",4.0,1.0,Analytics,170000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
21640,5/27/2020 18:52:14,Rakuten,BBB,Data Scientist,65000,"Tokyo, TY, Japan",2.0,0.0,ML / AI,43000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
55654,6/25/2021 22:52:52,Netflix,Senior Data Scientist,Data Scientist,525000,"Los Gatos, CA",6.0,2.0,Product,525000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
17052,2/18/2020 9:01:24,Spotify,Data Scientist,Data Scientist,173000,"New York, NY",5.0,1.0,Data science,145000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3485,1/2/2019 8:57:58,Capital One,Director,Data Scientist,235000,"Toronto, ON, Canada",9.0,7.0,ML / AI,180000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
47681,4/9/2021 11:53:04,Ciklum,Senior,Data Scientist,72000,"Kiev, KC, Ukraine",7.0,7.0,Machine Learning,72000.0,...,1,0,0,0,1,0,0,0,White,PhD
12746,11/5/2019 14:34:07,Amazon,L5,Data Scientist,150000,"Seattle, WA",3.0,1.0,ML / AI,115000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
5785,4/14/2019 17:17:24,LinkedIn,Data Scientist,Data Scientist,242000,"Sunnyvale, CA",1.0,0.0,ML / AI,155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [199]:
year_level_df['yearsofexperience'].value_counts()

2.0     351
5.0     343
3.0     314
4.0     301
1.0     214
0.0     190
6.0     178
7.0     141
10.0    132
8.0     124
9.0      64
15.0     45
12.0     42
11.0     37
13.0     25
20.0     21
16.0     18
14.0     15
17.0      6
18.0      4
25.0      4
23.0      2
22.0      2
19.0      1
26.0      1
30.0      1
21.0      1
34.0      1
Name: yearsofexperience, dtype: int64

In [162]:
# salary_outliers = year_level_df[year_level_df['totalyearlycompensation']>=800000]
# salary_outliers

In [200]:
clean_df= year_level_df.drop(columns=['timestamp', 'level', 'yearsatcompany', 'otherdetails','Highschool', 'Some_College', 
                               'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',
                              'Race_Hispanic', 'Race', 'Education','cityid','dmaid','rowNumber','tag','bonus','stockgrantvalue','basesalary','title'])
clean_df.head()
#get_dummies/oneHotEncoder on company size and employment type

,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,Male,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,Male,0,0,0
444,ebay,180000,"San Jose, CA",10.0,Female,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,Male,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,Male,0,0,0


In [201]:
# salary_outliers = clean_df[clean_df['totalyearlycompensation']>=800000]
# salary_outliers

In [202]:
import re
form_state_one = r'(\w+\s)*(\w+),\s([A-Z]{2})$'
matches_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False)
no_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False).sum()
print(no_form_one)

form_state_two = r'(\w+\s)*(\w+),\s([A-Z]{2}),\s(\w+)'
matches_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False)
no_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False).sum()
print(no_form_two)


clean_df.location[~matches_form_one & ~matches_form_two]

2226
351


/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


9061    Tel Aviv, Israel
Name: location, dtype: object

In [203]:
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,Male,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,Male,0,0,0
444,ebay,180000,"San Jose, CA",10.0,Female,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,Male,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,Male,0,0,0


In [204]:
def US_location_to_state(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        state = re.sub('(\w+\s)*(\w+),\s','', city)
        
        # return state
        return state
    # otherwise, return NaN
    else:
        return np.nan
    

def location_to_city(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        city = re.sub(',\s([A-Z]{2})$','', city)
        
        # return state
        return city
    # otherwise, return NaN
    else:
        return np.nan
clean_df = clean_df.loc[clean_df.location != "Tel Aviv, Israel"]
clean_df


,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,Male,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,Male,0,0,0
444,ebay,180000,"San Jose, CA",10.0,Female,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,Male,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,Male,0,0,0
...,...,...,...,...,...,...,...,...
62240,IBM,155000,"Washington, DC",5.0,Male,0,0,0
62283,Facebook,150000,"Singapore, SG, Singapore",0.0,Female,0,0,0
62285,ADP,185000,"Roseland, NJ",8.0,Male,0,0,0
62529,Google,685000,"Kirkland, WA",22.0,Male,0,0,0


In [205]:
clean_df['state'] = clean_df.location.str.extract(f'({form_state_one}|{form_state_two})', flags=re.IGNORECASE)[0].apply(US_location_to_state)

In [206]:
clean_df = clean_df[clean_df['state'].notna()]
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,LinkedIn,233000,"San Francisco, CA",4.0,Male,0,0,0,CA
440,Microsoft,218000,"Seattle, WA",11.0,Male,0,0,0,WA
444,ebay,180000,"San Jose, CA",10.0,Female,0,0,0,CA
454,Twitter,500000,"San Francisco, CA",4.0,Male,0,0,0,CA
495,Facebook,370000,"Seattle, WA",8.0,Male,0,0,0,WA


In [207]:
clean_df['city'] = clean_df.location.str.extract(f'({form_state_one})', flags=re.IGNORECASE)[0].apply(location_to_city)
clean_df.drop('location', axis=1, inplace=True)
clean_df.head()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,company,totalyearlycompensation,yearsofexperience,gender,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,LinkedIn,233000,4.0,Male,0,0,0,CA,San Francisco
440,Microsoft,218000,11.0,Male,0,0,0,WA,Seattle
444,ebay,180000,10.0,Female,0,0,0,CA,San Jose
454,Twitter,500000,4.0,Male,0,0,0,CA,San Francisco
495,Facebook,370000,8.0,Male,0,0,0,WA,Seattle


In [208]:
clean_df.drop(columns=['gender'], inplace=True)
clean_df.nunique()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


company                    397
totalyearlycompensation    375
yearsofexperience           28
Masters_Degree               2
Bachelors_Degree             2
Doctorate_Degree             2
state                       39
city                       159
dtype: int64

In [209]:
print(clean_df["company"].value_counts().head(10))

Amazon          224
Microsoft       202
Facebook        175
Google           83
Apple            75
Uber             60
IBM              58
Capital One      44
LinkedIn         34
Walmart Labs     33
Name: company, dtype: int64


In [173]:
# #Binning he companies- .cut with 4 bins...do we want to specifically design these?
# bin_labels = ['Small', 'Medium', 'Large']
# bins3 = clean_df['company'].value_counts()
# company_bins = pd.cut(bins3, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
# company_bins

In [174]:
#Binning he companies- .cut with 4 bins...do we want to specifically design these?
bin_labels = ['Small', 'Medium', 'Large', 'Very Large']
bins4 = clean_df['company'].value_counts()
company_bins = pd.cut(bins4, bins=4, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
company_bins


['Very Large',
 'Very Large',
 'Very Large',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 

In [175]:
#change company name to bin name
companies = []
for company in clean_df['company']:
    if company == "Amazon" or company == "Microsoft" or company == "Facebook":
        company = "Large"
    elif company == "Google" or company == "Apple" or company == "IBM" or company=="Uber":
        company = "Medium"
    else:
        company = "Small"
    companies.append(company)

In [176]:
#reassign company column to bin name
clean_bins_df = clean_df.copy()
clean_bins_df['company'] = companies
# clean_bins_df.drop(columns=['Doctorate_Degree'], inplace=True)
# clean_bins_df.drop(columns=['city'], inplace=True)
clean_bins_df.head(10)
# clean_bins_df['gender'].value_counts()

,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,Small,233000,Early Career,0,0,0,CA,San Francisco
440,Large,218000,Senior,0,0,0,WA,Seattle
444,Small,180000,Senior,0,0,0,CA,San Jose
454,Small,500000,Early Career,0,0,0,CA,San Francisco
495,Large,370000,Mid level,0,0,0,WA,Seattle
499,Large,200000,Early Career,0,0,0,WA,Seattle
509,Large,340000,Senior,0,0,0,WA,Bellevue
510,Medium,690000,Senior,0,0,0,WA,Kirkland
513,Small,600000,Early Career,0,0,0,CA,Los Gatos
523,Small,168000,Mid level,0,0,0,CA,Palo Alto


In [177]:
# salary_outliers = clean_bins_df[clean_bins_df['totalyearlycompensation']>=800000]
# salary_outliers

,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
4204,Small,900000,Senior,0,0,0,CA,San Jose
34136,Small,800000,Senior,0,0,0,CA,San Francisco
52222,Small,1000000,Senior,0,0,0,CA,San Francisco


In [106]:
# #is there linearity between location numerical representation and salary?
# clean_bins2 = clean_bins_df.drop(['location'], axis = 1)
# locations = clean_bins_df['location']
# clean_bins2.head()

In [107]:
# #Create label Encoder and encode categorical
# from sklearn.preprocessing import LabelEncoder
# lb_make = LabelEncoder()
# clean_bins2['company']= lb_make.fit_transform(clean_bins2['company'])
# clean_bins2['gender']= lb_make.fit_transform(clean_bins2['gender'])

# clean_bins2.head()

In [108]:
# clean_bins2['gender'].value_counts() #what do we do with 4 genders?

In [109]:
# #OneHot to new categories
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(sparse=False)
# company_encoded = lb_make.fit_transform(clean_bins2['company'])
# company_encoded = company_encoded.reshape(len(company_encoded), 1)
# onehot_company_encoded = enc.fit_transform(company_encoded)
# print(onehot_company_encoded)

In [178]:
#skip procedural encoding with get_dummies()?
# clean_dummies = clean_dummies.drop(['gender_2'], axis=1)
clean_dummies = pd.get_dummies(data=clean_bins_df, columns=['company', 'city', 'state', 'yearsofexperience'], drop_first=True)#drop the gender columns?

clean_dummies.head(15)

,totalyearlycompensation,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Medium,company_Small,city_Alexandria,city_Alpharetta,city_Ann Arbor,city_Annapolis Junction,...,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,yearsofexperience_Entry Level,yearsofexperience_Mid level,yearsofexperience_Senior
419,233000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,218000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
444,180000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
454,500000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,370000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
499,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
509,340000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
510,690000,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
513,600000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
523,168000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [179]:
salary_outliers = clean_dummies[clean_dummies['totalyearlycompensation']>=800000]
salary_outliers

,totalyearlycompensation,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Medium,company_Small,city_Alexandria,city_Alpharetta,city_Ann Arbor,city_Annapolis Junction,...,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI,yearsofexperience_Entry Level,yearsofexperience_Mid level,yearsofexperience_Senior
4204,900000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
34136,800000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52222,1000000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [180]:
X = clean_dummies.drop(columns=["totalyearlycompensation"])
y = clean_dummies["totalyearlycompensation"]
X.shape
# Scale the dataset using MinMaxScaler()
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [181]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    y, 
                                                    random_state=42)

In [182]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.45739905338302533
Testing Score: -1.7997764380573476e+24


In [183]:
#LinReg basically could not do better than true random... testing random Forest Regressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=42)

In [184]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 42)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor(n_estimators=200, random_state=42)

In [185]:
Y_pred = regressor.predict(X_test)  # test the output by changing values


In [186]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print(f"R2 Score: {sklearn.metrics.r2_score(y_test, Y_pred)}")

Training Score: 0.6296442453999975
Testing Score: 0.27923529428740435
R2 Score: 0.27923529428740435


In [118]:
# RFR model looks pretty good... 
#could we use cross validation and/or decision trees to determine which features are not contributing to the model
#then tune by removing these noise contributors

In [119]:
#neural network?- go back to clean_bins_df
# Generate our categorical variable lists
app_cats = clean_bins_df.dtypes[clean_bins_df.dtypes == "object"].index.tolist()
clean_bins_df[app_cats].nunique()

company                3
yearsofexperience      4
state                 39
city                 159
dtype: int64

In [120]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(clean_bins_df[app_cats]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cats)
encode_df.head()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,company_Large,company_Medium,company_Small,yearsofexperience_Early Career,yearsofexperience_Entry Level,yearsofexperience_Mid level,yearsofexperience_Senior,state_AL,state_AR,state_AZ,...,city_Walnut Creek,city_Waltham,city_Warren,city_Washington,city_Watertown,city_Wellesley,city_West McLean,city_Wilmington,city_Woonsocket,city_Worcester
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
# Merge one-hot encoded features and drop the originals
clean_bins_df = clean_bins_df.merge(encode_df, left_index=True, right_index = True)
clean_bins_df.drop(app_cats,1, inplace=True)
clean_bins_df.head()

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,totalyearlycompensation,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Large,company_Medium,company_Small,yearsofexperience_Early Career,yearsofexperience_Entry Level,yearsofexperience_Mid level,...,city_Walnut Creek,city_Waltham,city_Warren,city_Washington,city_Watertown,city_Wellesley,city_West McLean,city_Wilmington,city_Woonsocket,city_Worcester
419,233000,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,218000,0,0,0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
444,180000,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
454,500000,0,0,0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,370000,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
clean_bins_df.totalyearlycompensation.values

array([233000, 218000, 180000, 500000, 370000, 200000, 340000, 690000,
       600000, 168000, 187000, 225000, 220000, 130000, 112000, 200000,
       120000, 500000, 220000, 147000, 340000, 235000, 206000, 233000,
       149000, 356000, 305000, 200000, 215000, 700000, 243000, 260000,
       240000, 300000, 160000, 220000, 210000, 210000, 115000, 115000,
       188000, 215000, 315000, 240000, 230000, 216000, 125000, 322000,
       400000, 157000,  82000, 120000, 240000, 100000, 114000, 119000,
       250000, 182000, 120000, 318000, 135000, 228000, 137000, 156000,
       193000, 350000])

In [123]:
# Split our preprocessed data into our features and target arrays
y = clean_bins_df.totalyearlycompensation.values
X = clean_bins_df.drop(columns=['totalyearlycompensation'])
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [124]:
#  Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [125]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
# hidden_nodes_layer2 = 20
# hidden_nodes_layer3 = 14

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# #Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 10)                2090      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 2,101
Trainable params: 2,101
Non-trainable params: 0
_________________________________________________________________


In [126]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [127]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 


Epoch 1/50
2/2 [==============================] - 1s 3ms/step - loss: 65288.0260 - accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 8ms/step - loss: -43195.5262 - accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 3ms/step - loss: -92464.5807 - accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 21ms/step - loss: -352235.0938 - accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 6ms/step - loss: -381732.5000 - accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 10ms/step - loss: -350075.5521 - accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 31ms/step - loss: -290861.0156 - accuracy: 0.0000e+00
Epoch 8/50
2/2 [==============================] - 0s 2ms/step - loss: -208706.2708 - accuracy: 0.0000e+00
Epoch 9/50
2/2 [==============================] - 0s 2ms/step - loss: -453923.7292 - accuracy: 0.0000e+00
Epoch 10/50
2/2 [==============================

In [128]:
#RF?
#Create variables from df
# Split our preprocessed data into our features and target arrays
y = clean_bins_df.totalyearlycompensation.values
X = clean_bins_df.drop(columns=['totalyearlycompensation'])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [129]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/50
2/2 [==============================] - 0s 3ms/step - loss: 3745228.3333 - accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 0s 8ms/step - loss: 3841067.6667 - accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 8ms/step - loss: 3824006.5833 - accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 5ms/step - loss: 3874770.3333 - accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 4ms/step - loss: 3792318.2500 - accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 3ms/step - loss: 3717166.8333 - accuracy: 0.0000e+00
Epoch 7/50
2/2 [==============================] - 0s 2ms/step - loss: 3700023.8333 - accuracy: 0.0000e+00
Epoch 8/50
2/2 [==============================] - 0s 31ms/step - loss: 3639312.9167 - accuracy: 0.0000e+00
Epoch 9/50
2/2 [==============================] - 0s 14ms/step - loss: 3714597.1667 - accuracy: 0.0000e+00
Epoch 10/50
2/2 [===========================

In [130]:
# Stratified kfolds cross validation
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [131]:
# Feature Scaling for input features.
scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(X)

In [132]:
# Create  classifier object.
lr = linear_model.LogisticRegression()

In [133]:
#  Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
lst_accu_stratified = []

In [134]:
for train_index, test_index in skf.split(X, y):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    lr.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(lr.score(x_test_fold, y_test_fold))

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  UserWarning,


In [135]:
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

List of possible accuracy: [0.0, 0.0]

Maximum Accuracy That can be obtained from this model is: 0.0 %

Minimum Accuracy: 0.0 %

Overall Accuracy: 0.0 %

Standard Deviation is: 0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = clean_bins_df.totalyearlycompensation.values
X = clean_bins_df.drop(columns=['totalyearlycompensation'])

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

In [72]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
# cross validation
from sklearn.model_selection import cross_validate
from sklearn import datasets, linear_model
linreg = LinearRegression()
scoring = "balanced_accuracy"
linscores = cross_validate(linreg, X_train, y_train, scoring=scoring, return_estimator=True)

/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1953: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1953: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1953: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/e140563/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn

In [74]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.3934847902878551
Testing Score: -3.868237862268594e+26


In [75]:
#trying random forest with c-v
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

In [76]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 42)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor(n_estimators=200, random_state=42)

In [78]:
Y_pred = regressor.predict(X_test)  # test the output by changing values
Y_pred

array([325618.        , 254595.51190476, 203195.83333333, 239117.51388889,
       375757.20238095, 307982.58333333, 247275.47222222, 199290.83333333,
       192518.75      , 239117.51388889, 188848.33333333, 219096.25      ,
       228811.1031746 , 219096.25      ])

In [79]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.5048682801580273
Testing Score: -0.7927673138082811


In [190]:
clean_bins_new = clean_dummies.copy()
clean_bins_new = clean_bins_new[clean_bins_new['totalyearlycompensation']<800000]//got rid of the three outliers
# clean_bins_df['totalyearlycompensation'].nunique()
clean_bins_new['totalyearlycompensation'].nunique()

372

In [191]:
# Split our preprocessed data into our features and target arrays
y = clean_bins_new.totalyearlycompensation.values
X = clean_bins_new.drop(columns=['totalyearlycompensation'])

In [192]:
#trying random forest with c-v
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

In [193]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 42)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor(n_estimators=200, random_state=42)

In [194]:
Y_pred = regressor.predict(X_test)  # test the output by changing values
Y_pred

array([215153.89969604, 152407.67857143, 287437.59380099, 288043.94246032,
       399114.06042569, 287437.59380099, 182012.5       , 287437.59380099,
       236013.63888889, 108085.8756003 , 301020.72103175, 237640.97231119,
       141998.03186809, 158232.26154401, 263939.89484127, 161671.97619048,
       215484.35570388, 142068.33333333, 221295.48611111, 347579.85701798,
       164282.13956795, 564583.93127706, 173492.24513518, 223452.37900052,
       195951.01944036, 231314.25748789, 231314.25748789, 330323.45589133,
       139692.18163781, 231314.25748789, 126171.57142857, 416974.70630101,
       141998.03186809, 321389.66468254, 298630.0566628 , 214903.0487013 ,
       214903.0487013 , 138527.66666667, 228105.46517372, 229513.69322344,
       231314.25748789, 141998.03186809, 162310.13433788, 332474.34006924,
       270232.33333333, 153492.97879347, 307211.63780664, 242436.38816739,
       195761.6364799 , 213886.25      , 154527.79611325, 231314.25748789,
       229513.69322344, 1

In [195]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.6204916784634507
Testing Score: 0.39676920783566616


In [212]:
# dropping those improved score by 11%
#RFR without binning, drop gender
clean_df2 = clean_df.copy()
clean_dummies2 = pd.get_dummies(clean_df)
clean_dummies2

,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_3M,company_3m,company_7-Eleven,company_ABB,company_ADP,...,city_Walnut Creek,city_Waltham,city_Warren,city_Washington,city_Watertown,city_Wellesley,city_West McLean,city_Wilmington,city_Woonsocket,city_Worcester
419,233000,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
440,218000,11.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
444,180000,10.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
454,500000,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
495,370000,8.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62233,190000,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62240,155000,5.0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
62285,185000,8.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
62529,685000,22.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
# Split our preprocessed data into our features and target arrays
y = clean_dummies2.totalyearlycompensation.values
X = clean_dummies2.drop(columns=['totalyearlycompensation'])

In [214]:
#trying random forest with c-v
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

In [215]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 42)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor(n_estimators=200, random_state=42)

In [216]:
Y_pred = regressor.predict(X_test)  # test the output by changing values
Y_pred

array([152803.33333333, 149795.        , 264833.125     , 278965.33333333,
       303618.69047619, 275675.83333333, 154506.66666667, 208640.93452381,
       249668.0297619 , 127980.        , 224337.5       , 184296.58333333,
       144978.75      , 156007.41666667, 225175.        , 125860.        ,
       213573.5       , 132402.        , 225507.5       , 335488.33333333,
       183607.91666667, 300125.66666667, 186916.66666667, 256263.02930403,
       155164.99206349, 180804.58333333, 206812.58333333, 358861.52380952,
       132399.98809524, 208568.83333333, 138034.58333333, 460836.08333333,
       141582.08333333, 253312.08333333, 220021.79166667, 169479.66666667,
       183255.91666667, 135195.17857143, 214880.41666667, 208372.52777778,
       207405.08333333, 126065.        , 165929.91666667, 307352.4047619 ,
       234205.        , 150115.        , 228569.21428571, 199069.28571429,
       280217.77777778, 216551.33333333, 163790.        , 324490.08333333,
       206110.375     , 1

In [217]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)


print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.8771159003270766
Testing Score: 0.4409559362315587


In [139]:
# XGBoost?